<a href="https://colab.research.google.com/github/carlibeisel/mason_water_budget/blob/main/07_compile_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Compile data for Mason Water Budget**

By: Carli Beisel

Created: August 2024

**Variables:**

*qin* = streamflow at upstream gauge on reach

*qout* = streamflow at downstream gauge on reach

*sr* = surface runoff from agricultural irrigation

*gw* = groundwater

*evap* = potential open water evaporation

In [ ]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reach Water Budget Calculations & Data

In [ ]:
## --------------- ##
##  LOAD DATASETS  ##
## --------------- ##

# Streamflow
drainflow = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/drainflow_out/final_monthly_flow.csv')
drainflow['Month'] = pd.to_datetime(drainflow['Month'], format='%Y-%m')
drainflow['Reach'] = drainflow['Reach'].replace('Noble Drain', 'Noble Reach')

# Precipitation
precip = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/precip_out/reach_precip_monthly.csv')
precip['Reach'] = precip['Reach'].replace('Noble Drain', 'Noble Reach')

# Evaporation (PET)
pet = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/pet_out/reach_pet.csv')
pet['Month'] = pd.to_datetime(pet['Month'].astype(str) + '-2024', format='%m-%Y', errors='coerce')
pet['Reach'] = pet['Reach'].replace('Noble Drain', 'Noble Reach')

# Surface Runoff
surface_runoff = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/irrigation_runoff_out/reach_runoff_final.csv')
surface_runoff.rename(columns={'runoff(af)': 'SR (AF)'}, inplace=True)
surface_runoff['Reach'] = surface_runoff['Reach'].replace('Noble Drain', 'Noble Reach')


# Correct month formatting for surface runoff
month_mapping = {
    "jan": "01", "feb": "02", "mar": "03", "apr": "04",
    "may": "05", "jun": "06", "jul": "07", "aug": "08",
    "sep": "09", "oct": "10", "nov": "11", "dec": "12"
}
surface_runoff['Month'] = surface_runoff['month'].str.lower().map(month_mapping)
surface_runoff['Month'] = pd.to_datetime("2024-" + surface_runoff['Month'], format='%Y-%m', errors='coerce')

# TVGWFM groundwater
gw = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/groundwater_out/gw_final_kaf.csv')
gw['Reach'] = gw['Reach'].replace('Noble Drain', 'Noble Reach')
gw = gw[gw['Year'] == 2015]
gw.drop(columns=['Stress Period', 'Year'], inplace=True)
days_in_month = {
    1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
    7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
}
gw['Days in Month'] = gw['Month'].map(days_in_month)
gw['GW (AF)'] = (gw['Flow (cfperday)'] * gw['Days in Month']) / 43560
gw['Month'] = gw['Month'].apply(lambda x: f'2024-{int(x):02d}-01')
gw['Month'] = pd.to_datetime(gw['Month'], format='%Y-%m-%d')
gw = gw[['Reach', 'Month', 'GW (AF)']]


In [ ]:
## ------------------------------ ##
##   Merge into one .csv file (KAF)
## ------------------------------ ##

drainflow_selected = drainflow[['Reach', 'Month', 'Qin', 'Qout']]

pet_selected = pet[['Reach', 'Month', 'Reach Evap (AF)']]

precip_selected = precip[['Reach', 'Month', 'Total Monthly Precip (AF)']]
precip_selected['Month'] = pd.to_datetime(precip_selected['Month'], errors='coerce')

surface_runoff_selected = surface_runoff[['Reach', 'Month', 'SR (AF)']]
surface_runoff_selected['Reach'] = surface_runoff['Reach'].replace('Noble Drain', 'Noble Reach')

# Merge the DataFrames on "Reach" and "Month"
merged_df = drainflow_selected.merge(pet_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(precip_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(surface_runoff_selected, on=['Reach', 'Month'], how='outer')
merged_df = merged_df.merge(gw, on=['Reach', 'Month'], how='outer')

# Display the final merged DataFrame
merged_df.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget.csv')

In [ ]:
# ---------------------------- #
# Calculate Water Budget
# --------------------------- #

def calculate_wb_residual(df):
    """
    Calculate the water budget residual (Qout - Qin) in acre-feet (AF) for each Gauge at each Month.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Gauge', 'Month', 'Qin', and 'Qout' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'Flow Difference (AF)' column.
    """

    df['Qin (AF)'] = pd.to_numeric(df['Qin'], errors='coerce')
    df['Qout (AF)'] = pd.to_numeric(df['Qout'], errors='coerce')
    df['Precip (AF)'] = pd.to_numeric(df['Total Monthly Precip (AF)'], errors='coerce')
    df['Evap (AF)'] = pd.to_numeric(df['Reach Evap (AF)'], errors='coerce')
    df['SR (AF)'] = pd.to_numeric(df['SR (AF)'], errors='coerce')
    df['GW (AF)'] = pd.to_numeric(df['GW (AF)'], errors='coerce')

    # Calculate the flow difference ( Residual = output - input )
    df['Residual (AF)'] = (df['Qout (AF)'] + df['Evap (AF)']) - (df['Qin (AF)'] + df['Precip (AF)'])

    return df

final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget.csv')
residual = calculate_wb_residual(final_df)
residual = residual.drop_duplicates()
residual.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget.csv', index = False)

In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget.csv')
final_df


,Unnamed: 0,Reach,Month,Qin,Qout,Reach Evap (AF),Total Monthly Precip (AF),SR (AF),GW (AF),Qin (AF),Qout (AF),Precip (AF),Evap (AF),Residual (AF)
0,0,Noble Reach,2024-01-01,NaN,NaN,0.106427,0.454021,92.389697,-567.543604,NaN,NaN,0.454021,0.106427,NaN
1,1,Noble Reach,2024-02-01,NaN,NaN,0.233969,0.434065,94.890627,-499.677996,NaN,NaN,0.434065,0.233969,NaN
2,2,Noble Reach,2024-03-01,NaN,NaN,0.510852,0.379094,89.645064,-543.527712,NaN,NaN,0.379094,0.510852,NaN
3,3,Noble Reach,2024-04-01,359.901428,337.424884,0.772457,0.396310,138.287698,-699.467536,359.901428,337.424884,0.396310,0.772457,-22.100398
4,4,Noble Reach,2024-05-01,426.988902,849.099284,1.013721,0.255801,250.745399,-902.871598,426.988902,849.099284,0.255801,1.013721,422.868303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,67,Solomon Reach,2024-08-01,1034.282584,816.287903,0.920993,0.001654,273.885731,-2000.366165,1034.282584,816.287903,0.001654,0.920993,-217.075342
68,68,Solomon Reach,2024-09-01,820.185042,722.248958,0.664094,0.072499,203.261714,-1839.392402,820.185042,722.248958,0.072499,0.664094,-97.344489
69,69,Solomon Reach,2024-10-01,427.153311,411.676343,0.458239,0.071950,123.394269,-1563.918294,427.153311,411.676343,0.071950,0.458239,-15.090678
70,70,Solomon Reach,2024-11-01,71.082550,149.402788,0.181315,NaN,91.784895,-1308.726597,71.082550,149.402788,NaN,0.181315,NaN


In [ ]:
# -------------------------------------- #
# Create dataframe for Manuscript Tables
# -------------------------------------- #
final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/final_water_budget.csv')

reach_areas = {
    'Reach 1': 8.13,
    'Reach 2': 8.19,
    'Reach 3': 4.70,
    'Noble Reach': 2.61,
    'Purdum Reach': 7.07,
    'Solomon Reach': 2.22
}

def af_to_inches(row, col):
    area = reach_areas.get(row['Reach'])
    return (row[col] / area) * 12

AF_to_cfs = 59.502 #AF/month to cfs
thesis_df = pd.DataFrame()
thesis_df['Reach'] = final_df['Reach']
thesis_df['Month'] = final_df['Month']
thesis_df['Qin (cfs)'] = (final_df['Qin (AF)'] / AF_to_cfs).round(0)
thesis_df['Qout (cfs)'] = (final_df['Qout (AF)'] / AF_to_cfs).round(0)
thesis_df['Precip (in)'] = final_df.apply(lambda row: round(af_to_inches(row, 'Precip (AF)'), 1), axis=1)
thesis_df['Evap (in)'] = final_df.apply(lambda row: round(af_to_inches(row, 'Evap (AF)'), 1), axis=1)
thesis_df['Gains(cfs)'] = (final_df['Residual (AF)'] / AF_to_cfs).round(0)

thesis_df = thesis_df[['Reach', 'Month', 'Qin (cfs)', 'Precip (in)', 'Evap (in)', 'Qout (cfs)', 'Gains(cfs)']]

thesis_df.to_csv(
    '/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/thesis_table.csv',
    index=False,
    float_format='%.4f'
)

## Mason Drainshed Water Budget Calculations & Data

In [ ]:
# ------------ #
# Compile Data
# ------------ #

gw_mason = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/groundwater_out/mason_gw_final_kaf.csv')
gw = gw_mason[gw_mason['Year'] == 2015]
gw.drop(columns=['Stress Period', 'Year'], inplace=True)
days_in_month = {
    1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
    7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
}
gw['Days in Month'] = gw['Month'].map(days_in_month)
gw['GW (KAF)'] = (-gw['Flow (KAF)']) #flip the sign
gw['Month'] = gw['Month'].apply(lambda x: f'2024-{int(x):02d}-01')
gw['Month'] = pd.to_datetime(gw['Month'], format='%Y-%m-%d')
gw = gw[['Month', 'GW (KAF)']]


precip_mason = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/precip_out/mason_precip_monthly.csv')
precip_mason['Month'] = pd.to_datetime(precip_mason['Month'], errors='coerce')

runoff_mason = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/irrigation_runoff_out/mason_runoff_final.csv')
runoff_mason.rename(columns={'runoff(af)': 'SR (AF)'}, inplace=True)
month_mapping = {
    "jan": "01", "feb": "02", "mar": "03", "apr": "04",
    "may": "05", "jun": "06", "jul": "07", "aug": "08",
    "sep": "09", "oct": "10", "nov": "11", "dec": "12"
}
runoff_mason['Month'] = runoff_mason['month'].str.lower().map(month_mapping)
runoff_mason['Month'] = pd.to_datetime("2024-" + runoff_mason['Month'], format='%Y-%m', errors='coerce')

pet_mason = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/pet_out/mason_pet.csv')
pet_mason['Month'] = pd.to_datetime(pet_mason['Month'].astype(str) + '-2024', format='%m-%Y', errors='coerce')


flow_mason = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/drainflow_out/mason_final_monthly_flow.csv')
flow_mason['Month'] = pd.to_datetime(flow_mason['Month'], format='%Y-%m')
flow_mason.rename(columns={'Inflow (AF)': 'Qin (AF)'}, inplace=True)
flow_mason.rename(columns={'Outflow (AF)': 'Qout (AF)'}, inplace=True)


In [ ]:
## ------------------------------ ##
##   Merge into one .csv file (KAF)
## ------------------------------ ##

drainflow_selected = flow_mason[['Month', 'Qin (AF)', 'Qout (AF)']]

pet_selected = pet_mason[['Month', 'Evap (AF)']]

precip_selected = precip_mason[['Month', 'Precip(AF)']]

surface_runoff_selected = runoff_mason[['Month', 'SR (AF)']]

# Merge the DataFrames on "Reach" and "Month"
merged_df = drainflow_selected.merge(pet_selected, on=['Month'], how='outer')
merged_df = merged_df.merge(precip_selected, on=['Month'], how='outer')
merged_df = merged_df.merge(surface_runoff_selected, on=['Month'], how='outer')
merged_df = merged_df.merge(gw, on=['Month'], how='outer')

# Display the final merged DataFrame
merged_df.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/mason_final_water_budget.csv')

In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/mason_final_water_budget.csv')
final_df


In [ ]:
# ---------------------------- #
# Calculate Water Budget
# --------------------------- #

def calculate_wb_residual(df):
    """
    Calculate the water budget residual (Qout - Qin) in acre-feet (AF) for each Gauge at each Month.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Gauge', 'Month', 'Qin', and 'Qout' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'Flow Difference (AF)' column.
    """
    # Ensure 'Qin' and 'Qout' are numeric
    df['Qin (AF)'] = pd.to_numeric(df['Qin (AF)'], errors='coerce')
    df['Qout (AF)'] = pd.to_numeric(df['Qout (AF)'], errors='coerce')
    df['Precip (AF)'] = pd.to_numeric(df['Precip(AF)'], errors='coerce')
    df['Evap (AF)'] = pd.to_numeric(df['Evap (AF)'], errors='coerce')

    # Calculate the flow difference ( Residual = output - input )
    df['Residual (AF)'] = (df['Qout (AF)'] + df['Evap (AF)']) - (df['Qin (AF)'] + df['Precip (AF)'])

    return df

final_df = pd.read_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/mason_final_water_budget.csv')
residual = calculate_wb_residual(final_df)
residual = residual.drop_duplicates()
residual.to_csv('/content/drive/MyDrive/Data/Mason Water Budget/data_output/compiled_data/mason_final_residual.csv', index = False)